In [1]:
#Importing necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
import os
import joblib
import warnings
warnings.filterwarnings('ignore')

In [2]:
#importing both training excel data and test excel data
train_df=pd.read_excel('../data/Train_set.xlsx')
test_df=pd.read_excel('../data/Test_set.xlsx')

In [3]:
#Describing our datasets
DEBUG=False
if DEBUG:
    print(train_df.info())

In [4]:
train_df.head(5)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [5]:
test_df.head(5)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
0,Jet Airways,6/06/2019,Delhi,Cochin,DEL → BOM → COK,17:30,04:25 07 Jun,10h 55m,1 stop,No info
1,IndiGo,12/05/2019,Kolkata,Banglore,CCU → MAA → BLR,06:20,10:20,4h,1 stop,No info
2,Jet Airways,21/05/2019,Delhi,Cochin,DEL → BOM → COK,19:15,19:00 22 May,23h 45m,1 stop,In-flight meal not included
3,Multiple carriers,21/05/2019,Delhi,Cochin,DEL → BOM → COK,08:00,21:00,13h,1 stop,No info
4,Air Asia,24/06/2019,Banglore,Delhi,BLR → DEL,23:55,02:45 25 Jun,2h 50m,non-stop,No info


In [6]:
if DEBUG:
    print("Shape of Training data:",train_df.shape)#(10463, 31)
    print("Shape of Testing data:",test_df.shape)#(2645, 31)

Here we concatenate both data because we have to apply preprocessing for both train and test

This concatenated df is only used for preprocessing but not for the training data

In [7]:
df=pd.concat([test_df,train_df],ignore_index=True)

In [8]:
df['Airline'].unique()

array(['Jet Airways', 'IndiGo', 'Multiple carriers', 'Air Asia',
       'Air India', 'Vistara', 'SpiceJet', 'Vistara Premium economy',
       'GoAir', 'Multiple carriers Premium economy',
       'Jet Airways Business', 'Trujet'], dtype=object)

In [9]:
df.describe()#only price is numberical soo it describes only price

,Price
count,10683.000000
mean,9087.064121
std,4611.359167
min,1759.000000
25%,5277.000000
50%,8372.000000
75%,12373.000000
max,79512.000000


In [10]:
df.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
13349    False
13350    False
13351    False
13352    False
13353    False
Length: 13354, dtype: bool

In [11]:
#Dropping Duplicates
df.drop_duplicates(inplace=True)

In [12]:
#Replacing the null value in total stops by mode
df['Total_Stops'].fillna(df['Total_Stops'].mode()[0],inplace=True)

Now handling time and date

In [13]:
#1.For (Date_Of_Journey)
df['Date_of_Journey']=pd.to_datetime(df['Date_of_Journey'],format='%d/%m/%Y')
df['journey_day']=df['Date_of_Journey'].dt.day
df['journey_month']=df['Date_of_Journey'].dt.month
df['journey_year']=df['Date_of_Journey'].dt.year

In [14]:
df.drop(columns=['Date_of_Journey'],inplace=True,axis=1)

In [15]:
#Year is 2019   for all so removing it
print(df['journey_year'].unique())
df.drop(columns=['journey_year'],inplace=True)

[2019]


In [16]:
#Additional_Info doesnt play a significant role for output prediction soo dropping it
df['Additional_Info'].unique()
df.drop(columns=['Additional_Info'],axis=1,inplace=True)

In [17]:
#Now working with total-stops
df['Total_Stops'].unique()
#for this we replace or map them
df['Total_Stops'] = df['Total_Stops'].replace({
    'non-stop': 0,
    '1 stop': 1,
    '2 stops': 2,
    '3 stops': 3,
    '4 stops': 4
})


In [18]:
#working with duration
def duration_to_minutes_safe(duration):
    if pd.isnull(duration):
        return 0  # or np.nan if you prefer

    duration = duration.strip()
    hours = 0
    minutes = 0

    # Extract hours
    if 'h' in duration:
        hours_part = duration.split('h')[0].strip()
        if hours_part != '':
            hours = int(hours_part)

    # Extract minutes
    if 'h' in duration:
        mins_part = duration.split('h')[1].strip()
    else:
        mins_part = duration

    # Remove 'm' if present
    mins_part = mins_part.replace('m','').strip()
    if mins_part != '':
        minutes = int(mins_part)

    return hours*60 + minutes


df['Duration_in_time']=df['Duration'].apply(duration_to_minutes_safe)
df.drop(columns=['Duration'],inplace=True)

In [19]:
df

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Total_Stops,Price,journey_day,journey_month,Duration_in_time
0,Jet Airways,Delhi,Cochin,DEL → BOM → COK,17:30,04:25 07 Jun,1,NaN,6,6,655
1,IndiGo,Kolkata,Banglore,CCU → MAA → BLR,06:20,10:20,1,NaN,12,5,240
2,Jet Airways,Delhi,Cochin,DEL → BOM → COK,19:15,19:00 22 May,1,NaN,21,5,1425
3,Multiple carriers,Delhi,Cochin,DEL → BOM → COK,08:00,21:00,1,NaN,21,5,780
4,Air Asia,Banglore,Delhi,BLR → DEL,23:55,02:45 25 Jun,0,NaN,24,6,170
...,...,...,...,...,...,...,...,...,...,...,...
13349,Air Asia,Kolkata,Banglore,CCU → BLR,19:55,22:25,0,4107.0,9,4,150
13350,Air India,Kolkata,Banglore,CCU → BLR,20:45,23:20,0,4145.0,27,4,155
13351,Jet Airways,Banglore,Delhi,BLR → DEL,08:20,11:20,0,7229.0,27,4,180
13352,Vistara,Banglore,New Delhi,BLR → DEL,11:30,14:10,0,12648.0,1,3,160


In [20]:
#Working with arriaval time 
df['Arrival_Time']=pd.to_datetime(df['Arrival_Time'])
df['arrival_time_hour']=df['Arrival_Time'].dt.hour
df['arrival_time_minute']=df['Arrival_Time'].dt.minute
df['arrival_day']=df['Arrival_Time'].dt.day
df['arrival_month']=df['Arrival_Time'].dt.month

In [21]:
#Drop the Arrival_Time column
df.drop(columns=['Arrival_Time'],inplace=True,axis=1)

In [22]:
#Departure Time
df['Dep_Time']=pd.to_datetime(df['Dep_Time'])
df['Departure_hour']=df['Dep_Time'].dt.hour
df['Departure_minute']=df['Dep_Time'].dt.minute
df.drop(columns=['Dep_Time'],inplace=True,axis=1)

In [23]:
if DEBUG:
    print("-------Sources--------")
    print(df['Source'].value_counts())
    print("------Destinations-----")
    print(df['Destination'].value_counts())
    print('------Airlines-------')
    print(df['Airline'].value_counts())

In [24]:
#As we already have Column(Total_Stops) soo it isn't necessary to keep route column 
#soo dropping route column
df.drop(columns=['Route'],inplace=True,axis=1)

In [25]:
#We are going to use one hot encoding for both source and destination
if DEBUG:
    print('Sources are:',df['Source'].unique())
    print('Destination are:',df['Destination'].unique())

In [26]:
#Working for Airlines
#Here for 'Airlines' we use one hot encoding
if DEBUG:
    print(df['Airline'].unique())
    
"""['Jet Airways' 'IndiGo' 'Multiple carriers' 'Air Asia' 'Air India'
'Vistara' 'SpiceJet' 'Vistara Premium economy' 'GoAir'
'Multiple carriers Premium economy' 'Jet Airways Business' 'Trujet']"""

"['Jet Airways' 'IndiGo' 'Multiple carriers' 'Air Asia' 'Air India'\n'Vistara' 'SpiceJet' 'Vistara Premium economy' 'GoAir'\n'Multiple carriers Premium economy' 'Jet Airways Business' 'Trujet']"

Doing one hot encoding for : Airlines,Source,Destination

In [27]:
import pandas as pd

df = pd.get_dummies(
    df,
    columns=['Airline', 'Source', 'Destination'],
    drop_first=True,
    dtype=int
)

In [28]:
df.head(5)

,Total_Stops,Price,journey_day,journey_month,Duration_in_time,arrival_time_hour,arrival_time_minute,arrival_day,arrival_month,Departure_hour,...,Airline_Vistara Premium economy,Source_Chennai,Source_Delhi,Source_Kolkata,Source_Mumbai,Destination_Cochin,Destination_Delhi,Destination_Hyderabad,Destination_Kolkata,Destination_New Delhi
0,1,NaN,6,6,655,4,25,7,6,17,...,0,0,1,0,0,1,0,0,0,0
1,1,NaN,12,5,240,10,20,7,1,6,...,0,0,0,1,0,0,0,0,0,0
2,1,NaN,21,5,1425,19,0,22,5,19,...,0,0,1,0,0,1,0,0,0,0
3,1,NaN,21,5,780,21,0,7,1,8,...,0,0,1,0,0,1,0,0,0,0
4,0,NaN,24,6,170,2,45,25,6,23,...,0,0,0,0,0,0,1,0,0,0


In [29]:
df.isnull().sum()#For only testing dataset there are null values

Total_Stops                                     0
Price                                        2645
journey_day                                     0
journey_month                                   0
Duration_in_time                                0
arrival_time_hour                               0
arrival_time_minute                             0
arrival_day                                     0
arrival_month                                   0
Departure_hour                                  0
Departure_minute                                0
Airline_Air India                               0
Airline_GoAir                                   0
Airline_IndiGo                                  0
Airline_Jet Airways                             0
Airline_Jet Airways Business                    0
Airline_Multiple carriers                       0
Airline_Multiple carriers Premium economy       0
Airline_SpiceJet                                0
Airline_Trujet                                  0


In [34]:
df.columns

Index(['Total_Stops', 'Price', 'journey_day', 'journey_month',
       'Duration_in_time', 'arrival_time_hour', 'arrival_time_minute',
       'arrival_day', 'arrival_month', 'Departure_hour', 'Departure_minute',
       'Airline_Air India', 'Airline_GoAir', 'Airline_IndiGo',
       'Airline_Jet Airways', 'Airline_Jet Airways Business',
       'Airline_Multiple carriers',
       'Airline_Multiple carriers Premium economy', 'Airline_SpiceJet',
       'Airline_Trujet', 'Airline_Vistara', 'Airline_Vistara Premium economy',
       'Source_Chennai', 'Source_Delhi', 'Source_Kolkata', 'Source_Mumbai',
       'Destination_Cochin', 'Destination_Delhi', 'Destination_Hyderabad',
       'Destination_Kolkata', 'Destination_New Delhi'],
      dtype='object')

In [32]:
feature_columns = df.drop(columns=['Price']).columns
#Now saving models
models_path="../models"
joblib.dump(feature_columns,os.path.join(models_path,"preprocessed_columns.pkl"))

['../models\\preprocessed_columns.pkl']

In [35]:
#Now again seperating traning and testing data on the basic of the price(output feature)
train_df=df[df['Price'].notnull()]
test_df=df[df['Price'].isnull()]

In [36]:
#New shape after seperating
if DEBUG:
    print('Training_dataset shape:',train_df.shape)#(10463, 31)
    print('Testing_dataset shape:',test_df.shape)(2645, 31)

In [37]:
df.nunique()

Total_Stops                                     5
Price                                        1870
journey_day                                    10
journey_month                                   4
Duration_in_time                              374
arrival_time_hour                              24
arrival_time_minute                            12
arrival_day                                    19
arrival_month                                   5
Departure_hour                                 24
Departure_minute                               12
Airline_Air India                               2
Airline_GoAir                                   2
Airline_IndiGo                                  2
Airline_Jet Airways                             2
Airline_Jet Airways Business                    2
Airline_Multiple carriers                       2
Airline_Multiple carriers Premium economy       2
Airline_SpiceJet                                2
Airline_Trujet                                  2


Soo I am going to  create a new dataframe where there are no outliers and apply log transformation to target variable and use this to train my linear regression as the RandomForest and XGboost algorithm uses the same train_df and test_df

In [38]:
train_df_lr=train_df.copy()
test_df_lr=test_df.copy()
new_df_lr=pd.concat([train_df_lr,test_df_lr],ignore_index=True)

In [39]:
Q1 = train_df_lr['Price'].quantile(0.25)
Q3 = train_df_lr['Price'].quantile(0.75)
IQR=Q3-Q1
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR
train_df_lr = train_df_lr[(train_df_lr['Price'] >= lower) & (train_df_lr['Price'] <= upper)]

In [40]:
#Now we try to split the data for training out linear regression model
X=train_df_lr.drop(columns=['Price'])
y = np.log1p(train_df_lr['Price'])  # log-transform for LR
X_train_lr,X_value,y_train_lr,y_value=train_test_split(X,y,test_size=0.3,random_state=42)

In [41]:
#Now try to split the data for training our RandomForest and XGBoost algorithm
X_rf_xg=train_df.drop(columns=['Price'])
y_rf_xg=train_df['Price']
X_train_rf_xg,X_value_rf_xg,y_train_rf_xg,y_value_rf_xg=train_test_split(X_rf_xg,y_rf_xg,test_size=0.3,random_state=42)